In [1]:
import sys
import urllib.request
from PyQt5.QtWebEngineWidgets import QWebEnginePage
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QUrl
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
import bs4 as bs
import urllib.request
import copy
import re
import os.path
import json
import hashlib

class Page(QWebEnginePage):
    def __init__(self, url):
        self.app = QApplication(sys.argv)
        QWebEnginePage.__init__(self)
        self.html = ''
        self.loadFinished.connect(self._on_load_finished)
        self.load(QUrl(url))
        self.app.exec_()

    def _on_load_finished(self):
        self.html = self.toHtml(self.Callable)
        #print('Load finished:')

    def Callable(self, html_str):
        self.html = html_str
        self.app.quit()

def mergeToFile(hashID, processed, lines, justNameSyn):
    
    jsonHash = json.dumps(hashID)
    jsonProcessed = json.dumps(processed)
    jsonLines = json.dumps(lines)
    jsonJustNameSyn = json.dumps(justNameSyn)

    mergedResults = "HASH_ID " + jsonHash + " TEXT_FROM_CRAWLER "+ jsonProcessed + " TEXT_POG_TAGS " + jsonLines + " TEXT_SINGLE_SYNONYMS " + jsonJustNameSyn
    
    uploadToDesktop(mergedResults)
        
def uploadToDesktop(toFile):
    save_path = '/Users/bryanjordan/Sites/server_upload/crawler_nlp_results/'

    name_of_file = "five"

    completeName = os.path.join(save_path, name_of_file+".txt")         

    file1 = open(completeName, "w+")

    file1.write(toFile)

    file1.close()
    print("Complete")
    
def addToTotalText(hashID, text):
    save_path = '/Users/bryanjordan/Sites/server_upload/total_text/'
    
    name_of_file = "total_text_result"
    
    completeName = os.path.join(save_path, name_of_file+".txt")         

    file1 = open(completeName, "a+")
    
    uploadedText = "__BREAK__" + "\n" + hashID + "\n" + text + "\n"

    file1.write(uploadedText)

    file1.close()
    print("Uploaded to total_text")
    
        
def bs_preprocess(html):
    pat = re.compile('(^[\s]+)|([\s]+$)', re.MULTILINE)
    html = re.sub(pat, '', html)                                              
    html = re.sub('[\s]+<', '<', html) 
    html = re.sub('>[\s]+', '>', html) 
    return html     

def justSynonym(dictionary):
    
    indvSyns = {}

    for word in dictionary:
        
        individualResults = []
        
        result = dictionary[word]
        for synonym in result:
            temp = synonym.name().split(".")[0].replace('_',' ')
                        
            if not bool(individualResults):
                addValue = temp + " "
                individualResults = addValue

            if temp not in individualResults:
                addValue = temp + " "
                individualResults += addValue
                
        indvSyns[word] = individualResults
    return indvSyns

def filterWord(word):
    
    word = re.sub(r'[^\w]', '', word)
    
    return word

def findSynonyms(dictionary):
    
    synDict = {}
    
    for key in dictionary:
        if " " in key:
            temp = key.split()
            for word in temp:
                filtered = filterWord(word)
                syns = wordnet.synsets(filtered)
                synDict[filtered] = syns
        else:
            filtered = filterWord(key)
            syns = wordnet.synsets(filtered)
            synDict[filtered] = syns
            
    return synDict

def tagger(dictionary):
    
    for key in dictionary:
        if " " in key:
            temp = key.split()
            result = nltk.pos_tag(temp)
            dictionary[key] = result
        else:
            temp = filterWord(key)
            token = word_tokenize(temp)
            result = nltk.pos_tag(token)
            dictionary[key] = result
       
            
def main():
    page = Page('file:///Users/bryanjordan/Desktop/test.htm')
    soup = bs.BeautifulSoup(page.html, 'html5lib')
    js_test = soup.find('div', class_='settings_about_category')
    
    hash_object = hashlib.md5(b'settings_about_category')
    hashId = hash_object.hexdigest()
    
    result = js_test.text
    processed = bs_preprocess(result)
    
    lines = {}
    synonyms = {}
    
    #trim text per line
    for line in processed.splitlines():
        lines[line] = "TBC"
                
    #tags each word
    tagger(lines)
    
    #finds synonyms for each word
    synonyms = copy.deepcopy(lines)
    completeSynsForEachWord = findSynonyms(synonyms)
    justNameSyn = justSynonym(completeSynsForEachWord)
    addToTotalText(hashId, processed)
    mergeToFile(hashId, processed, lines, justNameSyn)    
    
if __name__ == '__main__': main()

Uploaded to total_text
Complete
